1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (input이 길면) 답변 생성이 오래 걸림
3. embedding -> vector database에 저장
4. 질문이 있을 때, vector database에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [ ]:
%pip install langchain_community docx2txt langchain_chroma langchainhub

In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./civil_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [8]:
len(document_list)

125

In [7]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [9]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(
#     documents=document_list, 
#     embedding=embedding,
#     collection_name='chroma-civil',
#     persist_directory='./chroma'
# )

# 이미 저장된 데이터를 사용할 때 
database = Chroma(
    collection_name='chroma-civil',
    persist_directory='./chroma',
    embedding_function=embedding
)

In [10]:
query = '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [14]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [15]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [16]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [17]:
ai_message = qa_chain({"query": query})

/Users/gonnim/.pyenv/versions/3.12.4/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [18]:
ai_message = qa_chain.invoke(query)

In [19]:
ai_message

{'query': '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?',
 'result': '부동산 매매 계약에서 소유 이전은 매도인이 매수인에게 매매의 목적이 된 권리를 이전하고, 매수인은 매도인에게 대금을 지급함으로써 이루어집니다. 이 의무는 특별한 약정이나 관습이 없으면 동시에 이행해야 합니다. 따라서 소유권 이전과 대금 지급은 상호 의무로 동시에 이루어지는 것이 원칙입니다.'}